In [1]:
import pandas as pd
import os

### Import Excel Sheets

In [2]:
file_path = "C:\\Users\\Rodrigues\\Desktop\\SQL Project-20250212\\Data.xlsx"
sheets = pd.read_excel(file_path, sheet_name=None)

In [3]:
df_costs = sheets['Costs']
df_budget = sheets['Budget']
df_items = sheets['Items']
df_currency = sheets['Currency']

### Remove an entry with non-existent Currency

In [4]:
df_costs = df_costs[df_costs['Currency'] != 'AOA']

df_costs.head()

,Date,% Item Key,% Key Facility,Warehouse,Account,Cost Center,Department,Order Nº,Order Type,To warehouse,WorkCenter,Amount,Currency
0,2015-11-10,1~I0202078,20.0,02G,402020.0,49100.0,2049100.0,2001065376,UIS,02G,NaN,1.20,GBP
1,2016-01-26,1~C0204766,20.0,02G,402020.0,49100.0,2049100.0,2001125029,UIS,02G,NaN,4.70,EUR
3,2015-04-02,1~I0200061,20.0,02G,433000.0,19000.0,2019000.0,2000054309,UIS,02G,NaN,2.35,USD
4,2015-04-20,1~I0202218,20.0,02G,433000.0,19000.0,2019000.0,2000060845,UIS,02G,NaN,2553.91,GBP
5,2015-07-13,1~I0202218,20.0,02G,433000.0,19000.0,2019000.0,2000110903,UIS,02G,NaN,2919.28,GBP


### Convert 'Date' columns to datetime format

In [5]:
df_currency['Date'] = pd.to_datetime(df_currency['Date'])
df_costs['Date'] = pd.to_datetime(df_costs['Date'])

df_currency = df_currency.sort_values(by=['Date'])
df_costs = df_costs.sort_values(by=['Date'])

C:\Users\Rodrigues\AppData\Local\Temp\ipykernel_7396\3026467828.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_costs['Date'] = pd.to_datetime(df_costs['Date'])


In [6]:
df_costs.head()

,Date,% Item Key,% Key Facility,Warehouse,Account,Cost Center,Department,Order Nº,Order Type,To warehouse,WorkCenter,Amount,Currency
21572,2014-03-05,1~C0208688,23.0,02G,552000.0,14000.0,2314000.0,2000000117,UIS,02G,23140,5.44,GBP
18914,2014-03-05,1~I0200030,23.0,02G,548020.0,40000.0,2340000.0,2000000165,UIS,02G,NaN,5.80,USD
18915,2014-03-05,1~I0200124,23.0,02G,548020.0,40000.0,2340000.0,2000000165,UIS,02G,NaN,3.80,EUR
18916,2014-03-05,1~I0200217,23.0,02G,548020.0,40000.0,2340000.0,2000000165,UIS,02G,NaN,5.22,USD
19274,2014-03-05,1~C0202820,23.0,02G,552000.0,11000.0,2311000.0,2000000093,UIS,02G,23110,3.54,USD


### Add Primary Key to Budget

In [7]:
df_budget["B_C_ID"] = range(1, len(df_budget) + 1)
df_budget.head()

,% Item Key,% Key Facility,Warehouse,Cost Center,Budget,B_C_ID
0,1~C0200000,23,02G,16000,190.858450,1
1,1~C0200000,25,02G,16000,260.855401,2
2,1~C0200001R,25,02G,16000,206.036554,3
3,1~C0200006,20,02G,51000,217.357675,4
4,1~C0200006,23,02G,13100,141.003067,5


In [8]:
duplicates = df_budget.duplicated(subset=['% Item Key', '% Key Facility', 'Warehouse', 'Cost Center'], keep=False)
df_duplicates = df_budget[duplicates]

if not df_duplicates.empty:
    print("Duplicate combinations found:")
    print(df_duplicates)
else:
    print("No duplicate combinations found. Each primary key is unique.")

No duplicate combinations found. Each primary key is unique.


### Add Foreign Key B_C_ID

In [9]:
df_costs = df_costs.merge(
    df_budget[['B_C_ID', '% Item Key', '% Key Facility', 'Warehouse', 'Cost Center']],
    how='left',  # Left join to keep all rows from df_costs
    left_on=['% Item Key', '% Key Facility', 'Warehouse', 'Cost Center'],
    right_on=['% Item Key', '% Key Facility', 'Warehouse', 'Cost Center']
)
df_costs.head()

,Date,% Item Key,% Key Facility,Warehouse,Account,Cost Center,Department,Order Nº,Order Type,To warehouse,WorkCenter,Amount,Currency,B_C_ID
0,2014-03-05,1~C0208688,23.0,02G,552000.0,14000.0,2314000.0,2000000117,UIS,02G,23140,5.44,GBP,12866.0
1,2014-03-05,1~I0200030,23.0,02G,548020.0,40000.0,2340000.0,2000000165,UIS,02G,NaN,5.80,USD,20455.0
2,2014-03-05,1~I0200124,23.0,02G,548020.0,40000.0,2340000.0,2000000165,UIS,02G,NaN,3.80,EUR,21267.0
3,2014-03-05,1~I0200217,23.0,02G,548020.0,40000.0,2340000.0,2000000165,UIS,02G,NaN,5.22,USD,22022.0
4,2014-03-05,1~C0202820,23.0,02G,552000.0,11000.0,2311000.0,2000000093,UIS,02G,23110,3.54,USD,4692.0


### Remove NaN from Costs

In [10]:
# Create a list of columns to check for NaN values (exclude 'WorkCenter')
columns_to_check = [col for col in df_costs.columns if col != 'WorkCenter']

# Remove rows where NaN values exist in any of the selected columns
df_costs = df_costs.dropna(subset=columns_to_check)
df_costs.head()

,Date,% Item Key,% Key Facility,Warehouse,Account,Cost Center,Department,Order Nº,Order Type,To warehouse,WorkCenter,Amount,Currency,B_C_ID
0,2014-03-05,1~C0208688,23.0,02G,552000.0,14000.0,2314000.0,2000000117,UIS,02G,23140,5.44,GBP,12866.0
1,2014-03-05,1~I0200030,23.0,02G,548020.0,40000.0,2340000.0,2000000165,UIS,02G,NaN,5.80,USD,20455.0
2,2014-03-05,1~I0200124,23.0,02G,548020.0,40000.0,2340000.0,2000000165,UIS,02G,NaN,3.80,EUR,21267.0
3,2014-03-05,1~I0200217,23.0,02G,548020.0,40000.0,2340000.0,2000000165,UIS,02G,NaN,5.22,USD,22022.0
4,2014-03-05,1~C0202820,23.0,02G,552000.0,11000.0,2311000.0,2000000093,UIS,02G,23110,3.54,USD,4692.0


In [11]:
print(df_costs['Date'].head())
print(df_costs['Date'].dtype)

0   2014-03-05
1   2014-03-05
2   2014-03-05
3   2014-03-05
4   2014-03-05
Name: Date, dtype: datetime64[ns]
datetime64[ns]


### Remove duplicates with the same Date and Currency

In [12]:
df_currency = df_currency.drop_duplicates(subset=['Date', 'Currency'], keep='last').copy()

### Add Primary Key to Currency

In [13]:
df_currency['C_C_ID'] = range(1, len(df_currency) + 1)
df_currency.head()

,Date,Currency,Currency Rate,C_C_ID
0,2007-01-01,EUR,1,1
1,2013-01-01,GBP,1,2
2,2013-01-01,PLN,"0,241127",3
3,2013-01-01,SEK,"0,120106",4
4,2013-09-01,CZK,"0,038665",5


In [14]:
duplicates = df_currency.duplicated(subset=['Date', 'Currency'], keep=False)
df_duplicates = df_currency[duplicates]

if not df_duplicates.empty:
    print("Duplicate combinations found:")
    print(df_duplicates)
else:
    print("No duplicate combinations found. Each primary key is unique.")

No duplicate combinations found. Each primary key is unique.


### Add Foreign Key C_C_ID

In [15]:
df_costs = pd.merge(df_costs, df_currency[['Date', 'Currency', 'C_C_ID']], on=['Date', 'Currency'], how='left')

df_costs.head()

,Date,% Item Key,% Key Facility,Warehouse,Account,Cost Center,Department,Order Nº,Order Type,To warehouse,WorkCenter,Amount,Currency,B_C_ID,C_C_ID
0,2014-03-05,1~C0208688,23.0,02G,552000.0,14000.0,2314000.0,2000000117,UIS,02G,23140,5.44,GBP,12866.0,922.0
1,2014-03-05,1~I0200030,23.0,02G,548020.0,40000.0,2340000.0,2000000165,UIS,02G,NaN,5.80,USD,20455.0,926.0
2,2014-03-05,1~I0200124,23.0,02G,548020.0,40000.0,2340000.0,2000000165,UIS,02G,NaN,3.80,EUR,21267.0,NaN
3,2014-03-05,1~I0200217,23.0,02G,548020.0,40000.0,2340000.0,2000000165,UIS,02G,NaN,5.22,USD,22022.0,926.0
4,2014-03-05,1~C0202820,23.0,02G,552000.0,11000.0,2311000.0,2000000093,UIS,02G,23110,3.54,USD,4692.0,926.0


In [16]:
nan_count = df_costs['C_C_ID'].isna().sum()

if nan_count > 0:
    print(f"There are {nan_count} NaN values in the 'C_C_ID' column.")
else:
    print("No NaN values found in the 'C_C_ID' column.")


There are 7352 NaN values in the 'C_C_ID' column.


### Remove NaN from Costs

In [17]:
nan_rows = df_costs[df_costs['C_C_ID'].isna()]

print(nan_rows)

             Date  % Item Key % Key Facility Warehouse   Account Cost Center  \
2      2014-03-05  1~I0200124           23.0       02G  548020.0     40000.0   
6      2014-03-05  1~C0206612           23.0       02G  552000.0     11000.0   
12     2014-03-05  1~C0205894           23.0       02G  552000.0     13100.0   
15     2014-03-05  1~C0212883           23.0       02G  552000.0     13300.0   
19     2014-03-05  1~C0202511           23.0       02G  552000.0     14000.0   
...           ...         ...            ...       ...       ...         ...   
101816 2016-02-06  1~C0207968           25.0       02G  552000.0     15000.0   
101817 2016-02-06  1~C0208042           25.0       02G  552000.0     15000.0   
101818 2016-02-06  1~C0202818           23.0       02G  552000.0     15000.0   
101819 2016-02-06  1~C0210972           24.0       02G  552000.0     15000.0   
101820 2016-02-06  1~C0201284           23.0       02G  559010.0     15000.0   

        Department    Order Nº Order Ty

In [18]:
df_currency['Date'] = pd.to_datetime(df_currency['Date'])
df_costs['Date'] = pd.to_datetime(df_costs['Date'])

df_currency = df_currency.sort_values(by=['Date'])
df_costs = df_costs.sort_values(by=['Date'])

df_temp = pd.merge_asof(  
    df_costs, 
    df_currency[['Date', 'Currency', 'C_C_ID']], 
    on='Date', 
    by='Currency',
    direction='backward' 
)

df_temp['C_C_ID'] = df_temp['C_C_ID_x'].ffill()

df_costs['C_C_ID'] = df_costs['C_C_ID'].fillna(df_temp['C_C_ID'])

print(df_costs.columns)  
print(f"Remaining NaN in C_C_ID: {df_costs['C_C_ID'].isna().sum()}")

Index(['Date', '% Item Key', '% Key Facility', 'Warehouse', 'Account',
       'Cost Center', 'Department', 'Order Nº', 'Order Type', 'To warehouse',
       'WorkCenter', 'Amount', 'Currency', 'B_C_ID', 'C_C_ID'],
      dtype='object')
Remaining NaN in C_C_ID: 0


In [19]:
nan_rows = df_costs[df_costs['C_C_ID'].isna()]

print(nan_rows)

Empty DataFrame
Columns: [Date, % Item Key, % Key Facility, Warehouse, Account, Cost Center, Department, Order Nº, Order Type, To warehouse, WorkCenter, Amount, Currency, B_C_ID, C_C_ID]
Index: []


### Export new excel

In [20]:
file_path = "C:\\Users\\Rodrigues\\Desktop\\SQL Project-20250212\\Data.xlsx"

directory = os.path.dirname(file_path)

new_file_path = os.path.join(directory, 'DataNew.xlsx')

with pd.ExcelWriter(new_file_path, engine='xlsxwriter') as writer:
    df_costs.to_excel(writer, sheet_name='Costs', index=False)
    df_budget.to_excel(writer, sheet_name='Budget', index=False)
    df_items.to_excel(writer, sheet_name='Items', index=False)
    df_currency.to_excel(writer, sheet_name='Currency', index=False)

print(f"Data successfully exported to {new_file_path}")

Data successfully exported to C:\Users\Rodrigues\Desktop\SQL Project-20250212\DataNew.xlsx
